In [ ]:
import pandas as pd
import pyodbc
import re

In [ ]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_reportedby,inc_notes from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC' and inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
df.head()

In [ ]:
roles = list(df['inc_reportedby'].value_counts().index)

In [ ]:
if '' in roles: roles.remove('')
print(roles)

In [ ]:
split_slash = []
for role in roles:
    for r in role.split('/'):
        split_slash.append(r.strip())
split_bracket = []
for role in split_dash:
    for r in role.split(' ('):
        split_bracket.append(r.strip(' )'))
role_search = []
for role in split_bracket:
    for r in role.split(' - '):
        role_search.append(r.strip(')'))
role_search = [role.replace('  ',' ') for role in role_search]
role_search = [role.lower().replace(' staff','') for role in role_search]
role_search = [role.strip(')') for role in role_search]
if 'unknown grade' in role_search: role_search.remove('unknown grade')
if 'all types' in role_search: role_search.remove('all types')
role_search = list(filter(lambda r: r!='other', role_search))

seen = set()
seen_add = seen.add
role_search = [x for x in role_search if not (x in seen or seen_add(x))]

role_search = [role.replace(' & ',' and ') for role in role_search]
role_search.append('dr')
role_search.append('1st year')
role_search.append('2nd year')
for i,role in enumerate(role_search):
    if re.sub(r"(\s)and(\s)",r"\1&\2",role)!=role: role_search.append(re.sub(r"(\s)and(\s)",r"\1&\2",role))
    #if re.sub(r"(\s)year",r"\1yr",role)!=role: role_search.append(re.sub(r"(\s)year",r"\1yr",role))
#for role in role_search:
#    if re.sub(r"administration",r"admin",role)!=role: role_search.append(re.sub(r"administration",r"admin",role))
role_search.remove('pharmacy')
print(role_search)

In [ ]:
role_num_dict = {}
for role in role_search:
    df_role = df[df['inc_notes'].str.lower().str.contains(' '+role+' ')]
    role_num_dict[role] = len(df_role)
    print(df_role['inc_notes'].values)

In [ ]:
print(role_num_dict)